In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader,Subset
from tqdm import tqdm
import pandas as pd
import sys

In [2]:
def make_dataset(augmentation_list):
    # 데이터 전처리 (CIFAR10은 3채널 이미지)
    train_transform = augmentation_list

    test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2023, 0.1994, 0.2010)
      )
    ])

    # CIFAR10 학습 및 테스트 데이터셋 불러오기
    train_dataset = datasets.CIFAR10(root='./data', train=True, transform=train_transform, download=True)
    test_dataset = datasets.CIFAR10(root='./data', train=False, transform=test_transform, download=True)

    # CIFAR10의 targets는 리스트로 되어 있으므로 tensor로 변환
    targets = torch.tensor(train_dataset.targets)

    num_samples_per_class = 2000
    selected_indices = []

    # 0부터 9까지 각 클래스별로 인덱스를 추출하고, 랜덤하게 num_samples_per_class개 선택
    for class_label in range(10):
        # 해당 클래스의 인덱스 추출
        indices = (targets == class_label).nonzero(as_tuple=True)[0]
        # 인덱스를 랜덤하게 섞은 후 원하는 개수만 선택
        selected = indices[torch.randperm(len(indices))[:num_samples_per_class]]
        selected_indices.extend(selected.tolist())

    # 선택한 인덱스만 남긴 서브셋 생성
    train_dataset = Subset(train_dataset, selected_indices)

    return train_dataset, test_dataset

In [3]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()

        self.features = nn.Sequential(
            # 1
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),

            # 2
            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            # 3
            nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # 4
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # 5
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),

            nn.Linear(in_features=256, out_features=2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),

            nn.Linear(in_features=2048, out_features=2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),

            nn.Linear(in_features=2048, out_features=10),
        )

    def forward(self, x):
        x = self.features(x)
        return x

In [4]:
def worker_init_fn(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

def run_experiment(augmentation_list, seed, i):
    # Ingredient 세팅
    names = ["None", "RandomCrop", "RandomHorizontalFlip", "Cutout", "All"]

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_dataset, test_dataset = make_dataset(augmentation_list)
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, pin_memory=True, num_workers=4, worker_init_fn=worker_init_fn)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, pin_memory=True, num_workers=4)

    model = AlexNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer = torch.optim.SGD(model.parameters(),lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40], gamma=0.2)

    num_epochs = 50

    # 시드 고정
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    print(f"[Seed : {seed} Experiment Start]\n")

    # Train
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}  Augmentation : {names[i]}', leave=True)
        for (data, target) in progress_bar:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            Loss = criterion(output, target)
            Loss.backward()
            optimizer.step()

            epoch_loss += Loss.item()
            progress_bar.set_postfix({"Loss": Loss.item()})

        scheduler.step()
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_loader)}, lr = {optimizer.param_groups[0]['lr']}")

    # Test
    model.eval()
    test_loss = 0.0
    correct = 0

    with torch.no_grad():
        for (data, target) in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            Loss = criterion(output, target)
            test_loss += Loss.item()
            correct += (output.argmax(dim=1) == target).sum().item()

    test_loss /= len(test_loader)
    test_accuracy = 100 * correct/len(test_loader.dataset)
    print(f"Test Loss : {test_loss}, Test Accuracy : {test_accuracy}%")

    return test_accuracy


In [5]:
# Augmentation 종류
mean=(0.4914,0.4822,0.4465); std=(0.2470,0.2435,0.2616)
augmentations = [
# 0: None
transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)]),

# 1: RandomCrop
transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.ToTensor(), transforms.Normalize(mean, std)]),

# 2: RandomHorizontalFlip
transforms.Compose([transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor(), transforms.Normalize(mean, std)]),

# 3: Cutout (RandomErasing)
transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.25), ratio=(1.0, 1.0), value=mean)
    ]),

# 4: ALL (RandomCrop + RandomHorizontalFlip + Cutout)
transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.25), ratio=(1.0, 1.0), value=mean)
    ])
]


def experiment_start():
    seed = np.random.randint(10000)
    result = []

    result.append(run_experiment(augmentations[0], seed, 0))
    result.append(run_experiment(augmentations[1], seed, 1))
    result.append(run_experiment(augmentations[2], seed, 2))
    result.append(run_experiment(augmentations[3], seed, 3))
    result.append(run_experiment(augmentations[4], seed, 4))

    return result

if __name__ == "__main__":
    torch.multiprocessing.set_start_method('fork', force=True)
    num_experiments = 3
    results = []

    for _ in range(num_experiments):
        results.append(experiment_start())

100%|██████████| 170M/170M [00:10<00:00, 16.2MB/s]


[Seed : 3367 Experiment Start]



Epoch 1/50  Augmentation : None: 100%|██████████| 40/40 [00:02<00:00, 14.84it/s, Loss=2.3]


Epoch 1, Loss: 2.3022349298000337, lr = 0.1


Epoch 2/50  Augmentation : None: 100%|██████████| 40/40 [00:01<00:00, 23.98it/s, Loss=2.04]


...

Epoch 49/50  Augmentation : All: 100%|██████████| 40/40 [00:02<00:00, 14.40it/s, Loss=0.588]


Epoch 49, Loss: 0.7936492756009101, lr = 0.004000000000000001


Epoch 50/50  Augmentation : All: 100%|██████████| 40/40 [00:02<00:00, 13.46it/s, Loss=0.738]

Epoch 50, Loss: 0.8032736271619797, lr = 0.004000000000000001


Test Loss : 0.9851335942745209, Test Accuracy : 67.33%


In [12]:
dfs = []
for i in range(len(results)):
    value = results[i]
    print(f"[Experiment {i} Result]")
    differences = [value[i] - value[0] for i in range(0, len(value))]

    index = [f"Accuracy", "Difference (with None)"]
    data = {
        "None" : [value[0], differences[0]],
        "RandomCrop" : [value[1], differences[1]],
        "RandomHorizontalFlip" : [value[2], differences[2]],
        "Cutout" : [value[3], differences[3]],
        "All" : [value[4], differences[4]]
    }

    df = pd.DataFrame(data, index=index)
    dfs.append(df)
    print(df, "\n\n")

avg_df = sum(dfs) / len(dfs)
print("Average")
avg_df

[Experiment 0 Result]
                         None  RandomCrop  RandomHorizontalFlip  Cutout    All
Accuracy                59.31       61.92                 62.69   61.51  66.60
Difference (with None)   0.00        2.61                  3.38    2.20   7.29 


[Experiment 1 Result]
                         None  RandomCrop  RandomHorizontalFlip  Cutout    All
Accuracy                59.09       65.41                 63.89   61.21  66.94
Difference (with None)   0.00        6.32                  4.80    2.12   7.85 


[Experiment 2 Result]
                         None  RandomCrop  RandomHorizontalFlip  Cutout    All
Accuracy                59.17       65.41                 63.59   61.17  67.33
Difference (with None)   0.00        6.24                  4.42    2.00   8.16 


Average


,None,RandomCrop,RandomHorizontalFlip,Cutout,All
Accuracy,59.19,64.246667,63.39,61.296667,66.956667
Difference (with None),0.00,5.056667,4.20,2.106667,7.766667
